In [5]:
import pandas as pd
#빌라주택, 원룸투룸  데이터 가져오기 
data = pd.read_csv('../preprocessing_data/최종/living_alone.csv')
data.head()
len(data)

98

In [141]:
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import time
from collections import defaultdict
from selenium import webdriver
import random

def circle_data_crawling(circle_id, circle_num):
    lst = []

    try:
        #아이디가 없는것도 있다 그런거는 스킵
        driver.find_element_by_xpath(f'//*[@id="{circle_id}"]').send_keys(Keys.ENTER)
    except:
        return []
    #밑으로 내려주기 내려주는거 횟수 저장하기 
    for _ in range(((circle_num // 20 )) * 5):
        itemlist = driver.find_element_by_xpath('//*[@id="listContents1"]/div')
        driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", itemlist)
        
        #일정한 시간에 클릭을 하니까 벤하는거 같다 
        time.sleep(0.5)
    
    time.sleep(0.5)
    piterpen = driver.page_source
    piterpen_soup = bs(piterpen, 'html.parser')
    piterpen_lst = []
    

    for i in range(1, circle_num + 1):
        if i in piterpen_lst: continue
        data = defaultdict(str)
        # number = random.uniform(0, 3)
        # time.sleep(number)
        #제목누르기 사진이 있는거와 없는거가 찾이가 난다 
        try:
            driver.find_element_by_xpath(f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a/div[1]/span').click()
            #사진 없음
            data['사진'] = 0
        except:
            driver.find_element_by_xpath(f'//*[@id="listContents1"]/div/div/div/div[{i}]/div/a[2]/div[1]/span').click()
            data['사진'] = 1

        time.sleep(0.5)
        estimate = driver.page_source
        estimate_soup = bs(estimate, 'html.parser')

        #피터팬이거나 중개소가 2곳인곳은 그냥 스킵
        try:
            detail_estimate = estimate_soup.find('div', class_ = 'detail_contents_inner')

            data['제목'] = detail_estimate.find('h4').text # 제목
            data['가격'] = detail_estimate.find('div', class_ ='info_article_price' ).text # 가격
            data['소제목'] = detail_estimate.find('div', class_ ='info_article_feature' ).text #소제목
            
            # 표 데이터 
            # 소재지가 없는 곳은 따로 좌표를 기준으로 동까지 추가해줘야겠다
            for i in detail_estimate.find('div', class_ = 'detail_box--summary').find_all('tr'):
                for title, content  in zip(i.find_all('th'), i.find_all('td')):
                    data[title.text] = content.text
        except:
            continue

        #데이터 저장
        lst.append(data)

    return lst

boodongsan_data = []
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
options.add_argument('--user-agent=' + user_agent)
driver = webdriver.Chrome('../chromedriver.exe', options= options)

for x, y in zip(data['x'], data['y']):
    driver.get(f'https://new.land.naver.com/rooms?ms={y},{x},19&a=APT:OPST:ABYG:OBYG:GM:OR:VL:DDDGG:JWJT:SGJT:HOJT&e=RETAIL&aa=SMALLSPCRENT&ad=true')
    time.sleep(3)
    page = driver.page_source
    soup = bs(page, 'html.parser')
    circle = soup.find('div',class_ = 'map_area').select('div > div > a')
    print(f'좌표 : {y},{x}, 원 갯수 {len(circle)}')
    for i in circle:
        try:
            circle_id = i['id']
            circle_num = i.select_one('span.sale_number').text.split('개')[0]
            boodongsan_data.extend(circle_data_crawling(circle_id,int(circle_num)))
            print(len(boodongsan_data))
        except:
            break
        

boodongsan_data = pd.DataFrame(boodongsan_data)
#데이터 저장
boodongsan_data.to_csv('../final_data/boodongsan_data_utf.csv', index=False, encoding='utf-8')

좌표 : 37.5213956191,126.8963096106, 원 갯수 21
1
좌표 : 37.5261691726,126.8913380449, 원 갯수 9
1
5
8
좌표 : 37.5246751205,126.9018412413, 원 갯수 12
10
10
21
21
21
21
21
21
좌표 : 37.5310404183,126.9012504769, 원 갯수 5
22
좌표 : 37.5317243396,126.905976257, 원 갯수 14
24
57
57
66
68
68
70
89
89
109
좌표 : 37.5619444748,126.9668582538, 원 갯수 9
110
118
120
120
120
좌표 : 37.5568320613,126.9657469404, 원 갯수 8
121
122
122
122
좌표 : 37.5573884114,126.9666888265, 원 갯수 7
123
123
124
좌표 : 37.5479129733,126.916106786, 원 갯수 11
125
127
131
131
131
133
133
좌표 : 37.552775759,126.9115986241, 원 갯수 22
133
135
136
138
138
141
141
148
151
161
187
188
190
191
191
195
200
200
좌표 : 37.5722683945,127.043011507, 원 갯수 5
200
좌표 : 37.5728684252,127.0435349, 원 갯수 7
201
205
207
좌표 : 37.5021887349,127.0482915806, 원 갯수 11
207
214
214
214
214
214
214
좌표 : 37.5021769677,127.0482912005, 원 갯수 11
214
221
221
221
221
221
221
좌표 : 37.5015660902,127.0488741832, 원 갯수 5
221
좌표 : 37.4859243711,126.9425199003, 원 갯수 34
222
224
268
270
271
304
305
306
306
3

,사진,제목,가격,소제목,소재지,매물특징,계약/전용면적,해당층/총층,방수/욕실수,월관리비,...,중개사,공급/전용면적,전세금 반환보증,용도,방구조,복층여부,현관구조,해당면적 세대수,사용검사일,준공인가일
0,0,상가주택5층,월세500/40,해당층/총층: 5/5층방 1개서향 계약/전용 면적:25.5㎡/17㎡,서울시 영등포구 당산동2가 10,올수리예정 대로변 엘리베이터 있는 옥탑,25.5㎡/17㎡(전용률67%),5/5층,1/1개,5만원,...,미래사랑공인중개사사무소길찾기대표이정은등록번호11560201800115소재지서울특별시...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,빌라 아테네 아테네5층,"전세2억 4,000(2,667만원/3.3㎡)",방 1개사용승인일:2016주차가능 공급/전용 면적:29.74㎡/19.84㎡,서울시 영등포구 양평동3가 49-4,남향 귀한전세 풀옵션 전세자금대출가능,NaN,5/7층,1/1개,4.5만원,...,우리공인중개사사무소(합동)길찾기대표김옥주등록번호92440000-08244소재지서울특...,29.74㎡/19.84㎡(전용률67%),"예상 보증료706,260원네이버페이 최대3만원 적립상품 안내",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,아테네 102동중층,"전세2억 3,000(1,807만원/3.3㎡)",남향역까지 8분 계약/전용 면적:42.08㎡/26.26㎡,NaN,1.5룸 남향 혼자 또는 신혼부주가능 전세자금대출가능,42.08㎡/26.26㎡(전용률62%),중/3층,1/1개,4.5만원,...,우리공인중개사사무소(합동)길찾기대표김옥주등록번호92440000-08244소재지서울특...,NaN,"예상 보증료676,840원네이버페이 최대3만원 적립상품 안내",주거용,분리형,단층,계단식,2세대,NaN,NaN
3,0,아테네 101동고층,"전세2억(1,536만원/3.3㎡)",남서향역까지 7분 계약/전용 면적:43.05㎡/26.9㎡,NaN,분리형 원룸 역세권 초특가 전세 현장,43.05㎡/26.9㎡(전용률62%),고/3층,1/1개,4만원,...,H공인중개사사무소대표이동식등록번호11560-2020-00153소재지서울특별시 영등포...,NaN,"예상 보증료588,550원네이버페이 최대3만원 적립상품 안내",-,오픈형,단층,복도식,2세대,NaN,NaN
4,0,빌라 아테네 101동고층,"전세2억 8,600(1,647만원/3.3㎡)",방 2개사용승인일:2016.07.19주차가능 공급/전용 면적:57.4㎡/38.04㎡,서울시 영등포구 양평동3가 49-4,마트 바로앞에 위치한 투룸,NaN,고/7층,2/1개,4.5만원,...,H공인중개사사무소대표이동식등록번호11560-2020-00153소재지서울특별시 영등포...,57.4㎡/38.04㎡(전용률66%),"예상 보증료841,630원네이버페이 최대3만원 적립상품 안내",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
#데이터 불러오기 
import pandas as pd
boodongsan_data = pd.read_csv('../final_data/boodongsan_data_utf.csv', encoding='utf-8')
boodongsan_data.head()

#데이터 전처리 

#소재지 없는거 제거 
dataframe = pd.DataFrame()

#결측치 제거 
boodongsan_data = boodongsan_data.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
print(len(boodongsan_data))
#소재지 없는곳 제거
boodongsan_data = boodongsan_data[~boodongsan_data['소재지'].isnull()]
print(len(boodongsan_data))

#보증금, 월세, 관리비 합치기 

887
684


In [193]:
import re
def deposit(x):
    if '(' in x:
        x = x.split('(')[0]
    #월세
    if '/' in x:
        x = x.split('/')[0]
    #숫자만 남기기 
    new_x = re.sub(r"[^0-9]", "", x)
    if int(new_x) < 10:
        new_x = int(new_x) * 10000
    
    return int(new_x)
def monthly(x):
    if '월세' in x:
        if '(' in x:
            x = x.split('(')[0]

        x = x.split('/')[1]
        x = re.sub(r"[^0-9]", "", x)
    else:
        x = 0
    
    return int(x)

boodongsan_data['보증금'] = boodongsan_data['가격'].apply(lambda x : deposit(x))
#전세이자 5%
boodongsan_data['보증금_5'] = boodongsan_data['보증금'] * 0.05
boodongsan_data['월세'] = boodongsan_data['가격'].apply(lambda x : monthly(x))

In [216]:
#전용면적 구하기 
boodongsan_data['계약/전용면적'].fillna('') + boodongsan_data['공급/전용면적'].fillna('')

0           25.5㎡/17㎡(전용률67%)
1       29.74㎡/19.84㎡(전용률67%)
4        57.4㎡/38.04㎡(전용률66%)
21           18㎡/18㎡(전용률100%)
22         39㎡/16.24㎡(전용률42%)
                ...          
1476          23㎡/21㎡(전용률91%)
1477    33.94㎡/26.14㎡(전용률77%)
1484    30.55㎡/25.47㎡(전용률83%)
1485    30.55㎡/25.47㎡(전용률83%)
1486    44.13㎡/29.96㎡(전용률68%)
Length: 684, dtype: object